## Import Libs

In [1]:
import os
import pickle
import json
import numpy as np
from sklearn.model_selection import KFold

from activation import *
from utils import *

## Load Data

In [2]:
train_path = os.path.join("data", "train.jsonl")
test_path = os.path.join("data","test.jsonl")

In [3]:
def fetch_data(train_path, test_path):
    with open(train_path, 'r') as f:
        train_data = [json.loads(line) for line in f]
    # Preprocess training data
    train_sentences = []
    train_labels = []
    for entry in train_data:
        tokens = entry['tokens']
        pos_tags = entry['pos_tags']
        chunk_tags = np.array(entry['chunk_tags'])
        
        train_sentences.append(pos_tags)
        train_labels.append(chunk_tags)

    with open(test_path, 'r') as f:
        test_data = [json.loads(line) for line in f]
    # Preprocess test data
    test_sentences = []
    test_labels = []
    for entry in test_data:
        tokens = entry['tokens']
        pos_tags = entry['pos_tags']
        chunk_tags = np.array(entry['chunk_tags'])
        
        test_sentences.append(pos_tags)
        test_labels.append(chunk_tags)
    
    return train_sentences, test_sentences,  train_labels, test_labels

X_train, X_test, y_train, y_test= fetch_data(train_path, test_path)


## Data Preprocessing

In [4]:
"""
Function to process the POS data
into One hot vectors,
that is :
Input: [DT/TT/NN/OT]    Shape: (1)
Output : [DT TT NN OT]  Shape: (4,1)
"""
def one_hot_encode(input_list):
    encoded_list = []
    for item in input_list:
        one_hot_vector = np.zeros(4)
        one_hot_vector[item - 1] = 1  # Adjust index to start from 0
        encoded_list.append(one_hot_vector.tolist())
    return np.array(encoded_list)

In [5]:
"""
Function to process the POS data
into Recurrent Perceptron input format,
that is :
Input : [DT TT NN OT]
"""
def into_ho(X_train):
    X_train_ho = []
    for i in range(len(X_train)):
        X = one_hot_encode(X_train[i])
        temp = []
        for j in range(len(X)):
            if j==0:
                temp.append(np.concatenate([np.array([1.0,0.0,0.0,0.0,0.0]), X[j]]))
            else:
                temp.append((np.concatenate([np.array([0]), X[j-1], X[j]] )))
        X_train_ho.append(np.array(temp))
    return X_train_ho

In [6]:
y_train[0]

array([1, 1, 1, 0, 1, 1, 1, 0, 1])

In [7]:
print(X_train[0])
one_hot_encode(X_train[0])

[1, 4, 3, 1, 4, 4, 3, 1, 4]


array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [8]:
X_train_ho = into_ho(X_train)
X_test_ho = into_ho(X_test)


## Recurrent Perceptron Model

In [29]:
"""
Some representation which will help understand this code base.
B       - Batch Size
Tx      - Length of Input string
X_i_b   -   
X_i_j   -   

"""

class SingleRecurrentPerceptron:
    def __init__(self, vec_len=10, lr=0.05):
          
        # Initialize weights and bias
        self.weights = np.random.randn(vec_len)
        self.threshold = np.random.randn(1)
        self.lr = lr

   
    def forward(self, inputs):
        """inputs-- (B, Tx, 10)"""   
        prediction= []    #(B, Tx)
        X_i_b = []      #(B, Tx, 10)
        for j in range(len(inputs)):
            out=[]
            X_i_j = []
            Tx, _ = inputs[j].shape
            y_prev=0
            for i in range(Tx):
                x = np.concatenate([inputs[j][i], np.array([y_prev])])
                X_i_j.append(x)
                net = x.T @ self.weights - self.threshold[0]
                oi = sigmoid(net)
                y_prev = oi
                out.append(oi)
            prediction.append(np.array(out))
            X_i_b.append(np.array(X_i_j))
        return X_i_b, prediction

    # def backward(self, inputs, target):
    #     """inputs-- (B, Tx, 10)
    #        target-- (B, Tx)
    #         """   
    #     X, prediction = self.forward(inputs)

    #     for i in range(len(inputs)):      # iterate over each example
    #         delta_w = np.zeros(10)
    #         for j in range(len(inputs[i])):     # iterate over each time
    #             x = X[i][j]
    #             delta_w += -self.lr * (target[i][j]-prediction[i][j]) * (x)
    #         self.weights += delta_w
    

            
    def backward_gpt(self, inputs, targets):

        X_i_b, prediction = self.forward(inputs)
         
        B = len(inputs)  # Get batch size, sequence length, and feature dim

        # Initialize gradients for weights and bias
        self.weights_grad = np.zeros_like(self.weights)
        self.threshold_grad = np.zeros_like(self.threshold)
        sequence_lengths = [len(t) for t in targets]

        # Calculate gradients for output layer (using element-wise multiplication)
        for b in range(B):
            Tx = sequence_lengths[b]
            for t in range(Tx):
                delta_t = (prediction[b][t] - targets[b][t]) * sigmoid_derivative(prediction[b][t])
                self.weights_grad += X_i_b[b][t]*delta_t
                self.threshold_grad += delta_t

            # BPTT (using chain rule)
            delta_prev = 0
            for t in reversed(range(Tx)):
                if t + 1 < Tx:
                    delta_t = delta_prev * sigmoid_derivative(prediction[b][t]) + np.dot(delta_t, self.weights)
                else:
                    delta_t = delta_prev * sigmoid_derivative(prediction[b][t])
                self.weights_grad -= X_i_b[b][t] * delta_t*0.3  # Exclude previous output
                delta_prev = delta_t

        # Normalize gradients by batch size
        self.weights_grad /= B
        self.threshold_grad /= B

        # Update weights and bias
        self.weights -= self.lr * self.weights_grad
        self.threshold -= self.lr * self.threshold_grad

    def calculate_loss(self, inputs, targets):
        
        """
        This function calculates the total loss for a minibatch of sequences.

        Args:
        inputs: Batch of input sequences (B, Tx_max, vec_len).
        targets: Batch of ground truth sequences (B, Tx_max).

        Returns:
        The average loss over the minibatch.
        """
        B = len(inputs)  # Get batch size, max sequence length, and feature dim

        # Initialize loss to zero
        loss = 0
        accuracy = 0
        _, predictions = self.forward(inputs)
        # Forward pass for each example in the minibatch
        for b in range(B):
            # Calculate loss per example using cross-entropy
          
            loss += cross_entropy_loss(predictions[b], targets[b])
            accuracy += np.mean(targets[b]==(predictions[b]>0.5).astype(int))
            
        # Average loss over the minibatch
        return loss / B, accuracy/B


        
    def train(self, inputs, targets, epochs):

        """inputs-- (B, Tx, 10)
           target-- (B, Tx)
            """           
        
        for iter in range(epochs):
            kf = KFold(n_splits=5, shuffle=True, random_state=42)
            train_loss = 0
            val_loss = 0
            train_accuracy = 0
            val_accuracy = 0
            for train_index, val_index in kf.split(inputs):
                train_inputs, val_inputs = [inputs[i] for i in train_index], [inputs[i] for i in val_index]
                train_targets, val_targets = [targets[i] for i in train_index], [targets[i] for i in val_index]
                self.backward_gpt(inputs, targets)
                delta_loss, delta_accuracy = self.calculate_loss(train_inputs,train_targets)
                train_loss += delta_loss
                train_accuracy += delta_accuracy
                delta_loss, delta_accuracy = self.calculate_loss(val_inputs,val_targets )

                val_loss += delta_loss
                val_accuracy += delta_accuracy
            print(f"epoch: {iter:.2f}, training loss : {train_loss/5:.2f}, training accuracy: {train_accuracy*100/5:.2f}%, validation loss: {val_loss/5:.2f}, validation accuracy: {val_accuracy*100/5:.2f}%")
    

    def save(self,path="model.pkl"):
        with open(path, 'wb') as f:
            pickle.dump(self, f)

    def load(self, path):
        with open(path, 'rb') as f:
            self = pickle.load(f) 
        return self

In [18]:
model = SingleRecurrentPerceptron(vec_len=10, lr=0.05)

In [19]:
model.train(X_train_ho, y_train, epochs=1)

epoch: 0.00, training loss : 208.31, training accuracy: 61.79%, validation loss: 208.31, validation accuracy: 61.79%


In [20]:
model.save(path="models/recurrent_perceptron.pkl")

In [21]:
model.weights

array([-0.52901238, -0.25053017,  0.55461461,  1.68287826,  0.30735051,
        1.11178686,  1.06321939,  0.93306754,  1.7354467 ,  0.08617973])

## Inference

In [32]:
model = SingleRecurrentPerceptron()
model = model.load("models/recurrent_perceptron.pkl")

In [22]:
tag_mapping = {1: "NN", 2: "DT", 3: "JJ", 4: "OT"}

# Calculate the number of unique POS tags
num_unique_tags = len(tag_mapping)

# Preprocess training data
# Preprocess training data
train_sentences = []
train_labels = []
for entry in train_data:
    tokens = entry['tokens']
    pos_tags = entry['pos_tags']
    chunk_tags = entry['chunk_tags']
    
    # Convert POS tags to one-hot encoded representation
    pos_tags_one_hot = np.zeros((len(pos_tags), num_unique_tags))
    for i, tag in enumerate(pos_tags):
        pos_tags_one_hot[i, tag - 1] = 1  # Subtract 1 to account for 0-based indexing
    
    # Flatten one-hot encoded representation
    flattened_tags = pos_tags_one_hot.flatten()
    
    train_sentences.append(flattened_tags)
    train_labels.append(chunk_tags)


# Initialize and train the single recurrent perceptron
# Initialize and train the single recurrent perceptron
input_size = len(train_sentences[0])  # Get input size from the first sample
output_size = 2  # Binary classification (1 for chunk, 0 for not chunk)
perceptron = SingleRecurrentPerceptron(input_size, output_size)
perceptron.train(train_sentences, train_labels, epochs=10)



# Load test data
with open('test.jsonl', 'r') as f:
    test_data = [json.loads(line) for line in f]

# Preprocess test data
test_sentences = []
test_labels = []
for entry in test_data:
    tokens = entry['tokens']
    pos_tags = entry['pos_tags']
    chunk_tags = entry['chunk_tags']
    
    # Convert POS tags to one-hot encoded representation
    pos_tags_one_hot = np.zeros((len(pos_tags), len(tag_mapping)))
    for i, tag in enumerate(pos_tags):
        pos_tags_one_hot[i, tag - 1] = 1  # Subtract 1 to account for 0-based indexing
    
    test_sentences.append(pos_tags_one_hot)
    test_labels.append(chunk_tags)

# Evaluate the trained perceptron
predictions = perceptron.predict(test_sentences)

# Assuming we have some evaluation function to compute accuracy
# Let's assume a simple accuracy calculation for demonstration
def accuracy(predictions, targets):
    correct = 0
    total = len(predictions)
    for pred, target in zip(predictions, targets):
        pred_labels = [1 if p > 0 else 0 for p in pred]
        if pred_labels == target:
            correct += 1
    return correct / total

acc = accuracy(predictions, test_labels)
print("Accuracy:", acc)

NameError: name 'train_data' is not defined